## Importação de Bibliotecas

In [32]:
# Python versão 3.8.8
# Windows 10

# bibliotecas para baixar arquivos e extrair arquivos zip
import urllib # versão 1.26.6
import urllib.request # versão 1.26.6
import zipfile # defaut version 20

# biblioteca para realizar requisição
import requests # versão 2.25.1

# biblioteca para interação com o sistema
import os
import shutil

# biblioeca para manipulação de dados
import pandas as pd # versão 1.3.2
import unidecode

# biblioteca para manipulação de banco de dados sqlite
import sqlite3 # versão 2.6.0
from sqlite3 import Error

import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='openpyxl')

import gzip

## Variáveis Globais

In [ ]:
path_atual = os.path.dirname(os.path.realpath('__file__')) + "\\"
nome_bd = "DW_Covid.sqlite"

## Download e Extração de Arquivoss

In [47]:
#Função para realizar o download de um arquivo através de uma url e salva-lo no diretório dado
def download_arquivo(url, path_arquivo, nome_arquivo = None):
    
    mensagem_download = "Realizando Download do Arquivo '{}'...".format(nome_arquivo)
    mensagem_sucesso = 'Arquivo {} salvo com sucesso.'.format(nome_arquivo).ljust(len(mensagem_download))

    print(mensagem_download, end="\r", flush=True)

    if not os.path.exists(path_arquivo):
        os.makedirs(path_arquivo)  # create folder if it does not exist

    if nome_arquivo is not None:
        filename = nome_arquivo
    else:
        filename = url.split('/')[-1].replace(" ", "_")  # be careful with file names

    file_path = os.path.join(path_arquivo, filename)

    r = requests.get(url, stream=True)
    if r.ok:
        print("saving to", os.path.abspath(file_path))
        with open(file_path, 'wb') as f:
            for chunk in r.iter_content(chunk_size=1024 * 8):
                if chunk:
                    f.write(chunk)
                    f.flush()
                    os.fsync(f.fileno())
    else:  # HTTP status code 4XX/5XX
        print("Download failed: status code {}\n{}".format(r.status_code, r.text))
        
        
# Função para extrair os metadados do zip (apaga o arquivo zip apos dar unzip)
def extrair_zip(path_arquivo, nome_arquivo):
        
    if (os.path.isfile(path_arquivo + nome_arquivo)):
        
        with gzip.open(path_arquivo + nome_arquivo, 'rb') as f_in:
            with open("tabela_casos_covid.csv", 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
                
        print("Metadados do COVID extraídos e salvos no diretório.", end="\n")
#         os.remove(path_arquivo + nome_arquivo)

In [26]:
url = "https://data.brasil.io/dataset/covid19/caso_full.csv.gz"
download_arquivo(url, path_atual, "casos_covid.gz")
extrair_zip(path_atual, "caso_full_csv.gz")

Arquivo casos_covid.gz salvo com sucesso.         


## Leitura de Dados

In [81]:
data_vacinacao = pd.read_excel("tabela_vacinacao_covid.xlsx")
data_casos = pd.read_csv("tabela_casos_covid.csv")

## Conexão Banco de Dados

In [88]:
def nova_conexao(path, nome_bd):

    conec_bd = None
    try:
        conec_bd = sqlite3.connect(path + nome_bd)

    except Error as erro:
        print(erro)
        return 0
    
    finally:
        if conec_bd:
            conec_bd.close()
    return 1

In [172]:
if (nova_conexao(path_atual,  nome_bd)):
    conec_bd = sqlite3.connect(path_atual + nome_bd)
    cursor = conec_bd.cursor()
    print("Conexão ao banco '{}' criada com sucesso.".format(nome_bd))
    conec_bd.commit()

Conexão ao banco 'DW_Covid.sqlite' criada com sucesso.


## Alimentação do Banco de Dados

In [94]:
def verifica_tabela_existe(cursor, nome_tabela):
    
    cursor.execute(''' SELECT name FROM sqlite_master WHERE type='table' AND name='{}' '''.format(nome_tabela))
    
    if cursor.fetchone():
        return 1
    else:
        return 0

In [151]:
def coverte_tipo_sqlite(string_tipo):
    
    if string_tipo == "object":
        tipo_sql = "text"
        
    elif string_tipo == "int64":
            tipo_sql = "integer"
            
    elif string_tipo == "float64":
            tipo_sql = "real"
    
    elif string_tipo == "bool":
            tipo_sql = "integer"            
    else:
        tipo_sql = "text"
    
    return tipo_sql
    

In [170]:
def insere_tabelas_banco(data, cursor, nome_tabela):

    tabela_existe = verifica_tabela_existe(cursor, nome_tabela)

    if not tabela_existe:


        # Transforma o nome da coluna seguido por seu tipo em apenas uma string para cada coluna
        # Logo após concatena essas strings utilizando a vírgula como separador
        lista_colunas = list(data.columns)
        
        novas_colunas = []
        for coluna in lista_colunas:
            coluna = coluna + " " + coverte_tipo_sqlite(str(data[coluna].dtype)) 
            novas_colunas.append(coluna)

        nomes_colunas = ", ".join(novas_colunas)         

        print("Criando tabela {} no banco {}...".format(nome_tabela, nome_bd), end="\r", flush=True)

        # Cria a tabela no banco de dados sqlite passando o nome da tabela, o nome das colunas e seus tipos
        comando = 'CREATE TABLE ' + nome_tabela + ' (' + nomes_colunas + ')'
        cursor.execute('' + comando + '')

        # Importa o dataframe com microdados referentes ao no para a tabela no banco de dados
        data.to_sql(nome_tabela, conec_bd, if_exists='append', index = False)
        conec_bd.commit()

        print(" "*70, end="\r", flush=True)
        print("Tabela {} criada com sucesso no banco {}.".format(nome_tabela, nome_bd), end="\n\n")

    # Se a tabela já existe, nada acontece
    else:
        print("Tabela {} já existe no banco de dados {}.".format(nome_tabela, nome_bd), end="\n\n")

# conec_bd.close()

In [173]:
insere_tabelas_banco(data_casos, cursor, "Casos_Covid")

CREATE TABLE Casos_Covid (city text, city_ibge_code real, date text, epidemiological_week integer, estimated_population real, estimated_population_2019 real, is_last integer, is_repeated integer, last_available_confirmed integer, last_available_confirmed_per_100k_inhabitants real, last_available_date text, last_available_death_rate real, last_available_deaths integer, order_for_place integer, place_type text, state text, new_confirmed integer, new_deaths integer)
Tabela Casos_Covid criada com sucesso no banco DW_Covid.sqlite.       



In [171]:
conec_bd.close()